<a href="https://colab.research.google.com/github/russellemergentai/MistralDocker/blob/main/TaskAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

T4 is fine.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from huggingface_hub import login
from google.colab import userdata

# Load the secret token
HF_TOKEN = userdata.get('HF_TOKEN')

# Check if the token is set and log in
if HF_TOKEN:
    login(HF_TOKEN)
    print("logged in to Hugging Face")
else:
    print("Hugging Face login failed")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
logged in to Hugging Face


In [2]:
!pip install -U bitsandbytes
!pip install torch transformers langchain
!pip install -U langchain-community
!pip install faiss-gpu-cu12

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

# load model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

#model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V5"
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left', add_eos_token=True, use_fast=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.7 MB/s eta 0:00:00


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [3]:
# create retriever
from langchain.document_loaders import TextLoader
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, SequentialChain
from google.colab import output
from langchain import PromptTemplate

import os
from pathlib import Path


def create_retriever(path):

    loader = TextLoader(path, encoding='UTF-8')

    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000, chunk_overlap=1000, separators=[" ", ",", "\n"]
    )

    texts = text_splitter.split_documents(documents)#
    model_path="intfloat/e5-large-unsupervised"

    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': False}
    )

    db = FAISS.from_documents(texts, embeddings)
    retriever = db.as_retriever()
    return retriever



def create_directory_retriever(directory_path):
    all_documents = []

    for file_path in Path(directory_path).rglob('*'):
        if file_path.is_file():
            loader = TextLoader(str(file_path), encoding='UTF-8')
            documents = loader.load()
            all_documents.extend(documents)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000, chunk_overlap=1000, separators=[" ", ",", "\n"]
    )

    texts = text_splitter.split_documents(all_documents)

    model_path = "intfloat/e5-large-unsupervised"

    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': False}
    )

    db = FAISS.from_documents(texts, embeddings)
    retriever = db.as_retriever()

    return retriever


# Define common pipeline parameters
common_params = {
    "torch_dtype": torch.bfloat16,
    "trust_remote_code": True,
    "device_map": {"": 0},
    "num_return_sequences": 1,
    "repetition_penalty": 1.5
}



# format output
def print_sections(result):
  words = result.split()
  for i in range(0, len(words), 12):
      chunk = words[i:i + 12]
      print(" ".join(chunk) + "\n")




# retrieve from data.txt or http page and run query
def retrieval_query_data():

  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
   <s>
  """

  path="/content/drive/MyDrive/Target/data.txt"

  retriever = create_retriever(path)
  output.clear()

  query = input("enter your query on the uploaded data: ")

  pipelineQuery = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    max_length=2000,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    **common_params,
    max_new_tokens=512
  )

  llmPipelineQuery = HuggingFacePipeline(pipeline=pipelineQuery, model_kwargs={"temperature": 0.1})
  qa = RetrievalQA.from_chain_type(llm=llmPipelineQuery, retriever=retriever, return_source_documents=False)
  result = qa.run({"query": query})

  del pipelineQuery
  del llmPipelineQuery
  del qa
  import gc
  gc.collect()

  #print_sections(result)
  print(result)



# query the base model with a commentary from the fine tuned model
def standard_query():
  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request. Keep your response brief.
  {query}
  [/INST]
  </s>
   <s>
  """

  query = input("enter the llm query: ")

  # Initialize pipelines with specific parameters
  pipelineBase = pipeline(
      "text-generation",
      model=llm,
      tokenizer=tokenizer,
      #max_length=500,
      #do_sample=True,
      eos_token_id=tokenizer.eos_token_id,
      **common_params,
      max_new_tokens=200
  )

  # Create HuggingFacePipeline objects
  llmPipeline = HuggingFacePipeline(pipeline=pipelineBase, model_kwargs={"temperature": 0.1})
  PROMPT = PromptTemplate.from_template(prompt_template)
  llm_chain = PROMPT | llmPipeline
  result = llm_chain.invoke({"query": query})

  del pipelineBase
  del llmPipeline
  del PROMPT
  del llm_chain
  import gc
  gc.collect()

  print_sections(result)


In [ ]:
# main loop
def main():
    functions = {'1': standard_query, '2': retrieval_query_data}

    while True:
        user_input = input("Enter 1 llm query, 2 data retrieval query or 'x' to exit: ").lower()
        output.clear()

        if user_input in functions:
            functions[user_input]()
        elif user_input == 'x':
            print("Exiting.")
            break
        else:
            print("Invalid input, please try again.")

if __name__ == "__main__":
    main()

enter your query on the uploaded data: when was hms victory built, specify the year


Device set to use cuda:0
<ipython-input-3-f63329d0c054>:123: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run({"query": query})
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=2000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

HMS Victory is a 104-gun first-rate ship of the line of the Royal Navy. She was ordered in 1758, laid down in 1759, and launched in 1765. With 246 years of service as of 2024, she is the world's oldest naval vessel still in commission.

Victory is best known for her role as Horatio Nelson's flagship at the Battle of Trafalgar on 21 October 1805. She had previously served as Keppel's flagship at Ushant, Howe's flagship at Cape Spartel and Jervis's flagship at Cape St Vincent. After 1824, she was relegated to the role of harbour ship. In 1922, she was moved to a dry dock at Portsmouth, England, and preserved as a museum ship. She has been the flagship of the First Sea Lord since October 2012.

On the morning of 21 October, the main British fleet, which was out of sight and sailing parallel some 10 miles away, turned to interce